In [163]:
import numpy as np
import matplotlib.pyplot as plt


In [164]:
# model parameters
dt = 1 # time step [s]
N = 4 # number of sections in the collumn
T = 5 # total simulation time steps

# sorbent parameters
K_initial = 1.5 # initial mass transfer coefficient [mmol/(m^2 s kPa)]
cap = 2.0 # cyclic uptake capacity [mol/L]

# process design parameters
l = 4 # collumn depth [m]
SSA = 210 # sorbent specific surface area [m^2/m^3]
# process operation parameters
v = 1.0 # air speed [m/s]
V_liq = 0.5 # liquid sorbent volume [L]




In [165]:
# derived parameters
l_N = l/N # length of one section [m]
V_N = l_N # volume of one section [m^3]
ds = v * dt / l_N # number of sections (that air travels in one time step)
ds_int_flag = ds.is_integer() # flag if ds is integer
#p = 2 # number of sections before inlet (for airspeeds larger than l_N/dt) -> could be set to np.ceil(ds)
p = int(np.ceil(ds)) # number of sections before inlet (for airspeeds larger than l_N/dt)
if ds_int_flag:
    print(f"ds = {ds} is integer.")
print(f"ds: {ds} [segments per timesteps], -> p: {p} [segments before inlet]")

# concentration matrix
x = np.ones((T, N+p)) * np.nan # [Pa]. rows: time steps; collumns: sections

# set initial and boundary conditions: 
# 1) x = 40 Pa for all time steps(=rows) at inlet (p initial sections=columns);
x[:, 0:p] = 40 # initial concentration in inlet [Pa]
# 2) x = 40 Pa at t=0 for all sections:columns
x[0, :] = 40 # initial concentration in collumn [Pa]
x

# mol in gase phase matrix (mol per time step and section)
n = np.ones((T, N)) * np.nan # rows: time steps; collumns: sections
# uptake matrix (mol per time step and section)
n2l = np.ones((T, N)) * np.nan # rows: time steps; collumns: sections

# total uptake (over time)
n_total_t = np.zeros(T) # total uptake per time step [mol]

ds = 1.0 is integer.
ds: 1.0 [segments per timesteps], -> p: 1 [segments before inlet]


In [166]:
x

array([[40., 40., 40., 40., 40.],
       [40., nan, nan, nan, nan],
       [40., nan, nan, nan, nan],
       [40., nan, nan, nan, nan],
       [40., nan, nan, nan, nan]])

In [167]:
def K_eff(uptake, K_initial, cap, V_liq):
    """Calculate effective mass transfer coefficient based on current uptake.
    
    Args:
        uptake (float): Current uptake in mol.
        K_initial (float): Initial mass transfer coefficient in mmol/(m^2 s kPa).
        cap (float): Cyclic uptake capacity in mol/L.
        V_liq (float): Liquid sorbent volume in L.
        N (int): Number of sections in the column.
        
    Returns:
        float: Effective mass transfer coefficient in mmol/(m^2 s kPa).
    """
    if uptake >= cap * V_liq:
        return 0.0
    else:
        return K_initial * (1 - uptake / (cap * V_liq))

In [ ]:
for t in range(0, T-1):
    # 1) calculate uptake n for section i at time step t
    n2l[t, :] = x[t,p:N+p] * K_initial * 1e-6 * SSA * l_N * dt # uptake gas molecules to liquid [mol]
    n[t, :] = x[t,p:N+p] * V_N / (8.314 * 298) - n2l[t, :]# remaining in gas phase [mol]
    if (n < 0.0).any(): # check if remaining in gas phase is negative
        print(f"Warning: negative remaining in gas phase! setting to zero. time step: {t}, index: {np.where(n < 0.0)}")
        n2l[t, n[t, :] < 0.0] = x[t,p:N+p][n[t, :] < 0.0] * V_N / (8.314 * 298) # limit uptake to remaining in gas phase
        n[t, n[t, :] < 0.0] = 0.0
    #n2l[t, :] = x[t,p:N+p] * K_initial * 1e-6 * SSA * l_N * dt * 44 # uptake [g_co2]

    # 2) calc remaining concentration after uptake:
    x[t, p:N+p] = n[t, :] * 8.314 * 298 / (V_N) # new concentration [Pa]
    
    # 3) calc next concentration x for all sections (p:N+p) at time step t+1
    if ds_int_flag:# if ds is int(ds)
        x[t+1, p:N+p] = x[t, (p-int(ds)):(N+p-int(ds))] # new concentration (in next time step) equals the one 

        





In [171]:
x

array([[4.00000000e+01, 4.23397471e-01, 4.23397471e-01, 4.23397471e-01,
        4.23397471e-01],
       [4.00000000e+01, 8.78259280e+00, 9.29631895e-02, 9.29631895e-02,
        9.29631895e-02],
       [4.00000000e+01, 8.78259280e+00, 1.92834841e+00, 2.04114460e-02,
        2.04114460e-02],
       [4.00000000e+01, 8.78259280e+00, 1.92834841e+00, 4.23397471e-01,
        4.48163546e-03],
       [4.00000000e+01, 4.00000000e+01, 8.78259280e+00, 1.92834841e+00,
        4.23397471e-01]])

In [ ]:
int(1.8)

1

In [ ]:
x[t, p-ds:N+p-ds]

TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:
n[t, :] * 8.314 * 298 / (V_N) #

array([nan, nan, nan, nan])

In [ ]:
n

array([[0.0126, 0.0126, 0.0126, 0.0126],
       [   nan,    nan,    nan,    nan],
       [   nan,    nan,    nan,    nan],
       [   nan,    nan,    nan,    nan],
       [   nan,    nan,    nan,    nan]])

In [ ]:
        # n[t, i-p] = K_initial * (x[t-1, i] - x[t-1, i-1]) * l_N / v * dt # uptake [mmol]
        # if n[t, i-p] > cap * V_liq / N: # check if uptake capacity is exceeded
        #     n[t, i-p] = cap * V_liq / N # limit uptake to capacity
        # x[t, i] = x[t-1, i] - n[t, i-p] / (l_N * 1e3) # new concentration [Pa]

In [ ]:
range(0, T)[T-1]

4